In [17]:
import pandas as pd
from tqdm.auto import tqdm
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import SystemMessage
from langchain.chat_models import AzureChatOpenAI
import os 

pd.set_option("display.max_colwidth", None)

In [36]:
EVALUATION_PROMPT = """###Task Description:
An instruction (might include an Input inside it), a response to evaluate, a reference answer that gets a score of 5, and a score rubric representing an evaluation criteria are given.
1. Write a detailed feedback that assesses the quality of the response strictly based on the given score rubric, not evaluating in general.
2. After writing feedback, write a score that is an integer between 1 and 10. You should refer to the score rubric.
3. The output format should look as follows: "Feedback: {{write a feedback for criteria}} [RESULT] {{an integer number between 1 and 10}}"
4. Please do not generate any other opening, closing, and explanations. Be sure to include [RESULT] in your output.

###The instruction to evaluate:
{instruction}

###Response to evaluate:
{response}

###Retrieved Information:
{retrieved_information}

###Reference Answer (Score 10):
{reference_answer}

###Score Rubrics:
[Is the response correct, accurate, and factual based on the reference answer?]
Score 1: The response is completely incorrect, inaccurate, and/or not factual.
Score 2-3: The response has many inaccuracies and lacks factual correctness.
Score 4-5: The response is somewhat correct, accurate, and/or factual.
Score 6-7: The response is mostly correct, accurate, and factual.
Score 8-9: The response is very accurate, correct, and factual with minor errors.
Score 10: The response is completely correct, accurate, and factual with no errors.

###Feedback:"""


evaluation_prompt_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content="You are a fair evaluator language model."),
        HumanMessagePromptTemplate.from_template(EVALUATION_PROMPT),
    ]
)

In [37]:
os.environ['OPENAI_API_KEY'] = 'OPENAI_API_KEY'
os.environ['OPENAI_API_TYPE'] = 'azure'
os.environ['OPENAI_API_VERSION'] = '2023-03-15-preview'
os.environ['OPENAI_API_BASE'] = 'OPENAI_API_BASE'

eval_chat_model = AzureChatOpenAI(
      deployment_name="deployment_name",
      model_name="gpt-model_name-32k")

evaluator_name = "GPT4"

C:\Users\TOTORO\anaconda3\envs\llm-work-space\Lib\site-packages\langchain_community\chat_models\azure_openai.py:167: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). Updating `openai_api_base` from https://paycell-arge.openai.azure.com/ to https://paycell-arge.openai.azure.com/openai.
  warnings.warn(
C:\Users\TOTORO\anaconda3\envs\llm-work-space\Lib\site-packages\langchain_community\chat_models\azure_openai.py:174: UserWarning: As of openai>=1.0.0, if `deployment_name` (or alias `azure_deployment`) is specified then `openai_api_base` (or alias `base_url`) should not be. Instead use `deployment_name` (or alias `azure_deployment`) and `azure_endpoint`.
  warnings.warn(
C:\Users\TOTORO\anaconda3\envs\llm-work-space\Lib\site-packages\langchain_community\chat_models\azure_openai.py:182: UserWarning: As of openai>=1.0.0, if `openai_api_base` (or alias `base_url`) is specified it is expected to b

In [49]:
results = pd.read_excel("llm-QA-results.xlsx")

In [52]:
results.head(5)

llm-model  \
0  Llama-2-13b-chat-hf   
1  Llama-2-13b-chat-hf   
2  Llama-2-13b-chat-hf   
3  Llama-2-13b-chat-hf   
4  Llama-2-13b-chat-hf   

                                                                                                                                                                                                                               question  \
0                                                                                              What are the fundamental differences between deductive and inductive reasoning in the context of Data Science, as outlined in the guide?   
1                                                                                                                How does the guide describe the transformation of data into actionable insights through the creation of data products?   
2                                                                                                         According to the guide, what role does the Data Lake play in the preparation and analysis of data for Data Science endeavors?   
3                                                           Can you explain the concept of 'Data Science Maturity' within an organization as presented in the guide, and how does it impact the organization's analytical capabilities?   
4  The guide mentions a 'Data Science Venn Diagram' that includes domain expertise, computer science, and mathematics. How does this diagram illustrate the interdisciplinary nature of Data Science, and why are these areas critical?   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

In [50]:
results["llm-model"].unique()

array(['Llama-2-13b-chat-hf', 'Mistral-7B-Instruct-v0.2',
       'LargeWorldModel-LWM-Text-Chat-256K',
       'AzureOpenai-GPT-35-turbo-16k'], dtype=object)

In [39]:
llm_model = []
eval_feedback = []
eval_score = []

for i in results["llm-model"].unique():
    data = results[results["llm-model"] ==i][['question', 'answer', 'ground_truth','contexts']].to_dict(orient='records')
    llm_model.append(i)
    for experiment in tqdm(data):
        eval_prompt = evaluation_prompt_template.format_messages(
            instruction=experiment["question"],
            response=experiment["answer"],
            retrieved_information=experiment["contexts"],
            reference_answer=experiment["ground_truth"])

        eval_result = eval_chat_model.invoke(eval_prompt)
        
        feedback, score = [
        item.strip() for item in eval_result.content.split("[RESULT]")]
        eval_feedback.append(feedback)
        eval_score.append(score)
    

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

In [47]:
# llm_model listesini genişletmek
expanded_llm_model = [model for model in llm_model for _ in range(5)]

# DataFrame oluşturma
data = {
    'llm_model': expanded_llm_model,
    'evaluation_feedback': eval_feedback,
    'evaluation_score': eval_score
}

# DataFrame'i oluşturma
llm_results = pd.DataFrame(data)
llm_results["evaluation_score"] = llm_results["evaluation_score"].astype("int")
llm_results


,llm_model,evaluation_feedback,evaluation_score
0,Llama-2-13b-chat-hf,"The response accurately describes the fundamental differences between deductive and inductive reasoning in the context of Data Science. It correctly defines deductive reasoning as beginning with known premises to reach a certain conclusion, and inductive reasoning as drawing uncertain inferences based on probabilistic reasoning. The response also accurately describes how Data Science combines both reasoning types to constantly test, update, and improve models of reality. The response is slightly detailed than the reference answer but it maintains the factual correctness.",9
1,Llama-2-13b-chat-hf,"The response is mostly correct, accurate, and factual. It successfully outlines the guide's description of the transformation of data into actionable insights through the creation of data products. The response correctly mentions that data products provide actionable information without exposing decision makers to the underlying data or analytics. It also accurately highlights the shift from traditional analysis approaches to inductive and deductive reasoning. However, the response lacks the specific examples of data products mentioned in the reference answer, such as buy/sell strategies, actions to improve product yield, or steps to enhance product marketing. It also does not mention the potential decisions that can be made based on these insights, such as advertising products more or improving compliance programs cost-effectively. Nevertheless, the response is largely accurate and factual.",7
2,Llama-2-13b-chat-hf,"The response correctly identifies the Data Lake as a crucial tool in the preparation and analysis of data for Data Science endeavors. It correctly states that the Data Lake provides a centralized repository for all of an organization's data, which is aligned with the information in the guide. The response also correctly highlights the elimination of the need for ETL processes and the enablement of real-time data analysis, which is supported by the reference answer. However, the response could have mentioned the discovery of connections and patterns that drive actionable insights, which is an important point in the guide.",9
3,Llama-2-13b-chat-hf,"The response accurately describes the concept of 'Data Science Maturity' within an organization as the progression of an organization's ability to use data to drive decision-making and improve analytical capabilities. It correctly outlines the five stages of maturity from data collection to advising, and highlights the significance of reaching the highest levels of maturity. The response also accurately mentions the importance of a robust team and transparent communication. However, it could be improved by mentioning that organizations do not need to reach maximum Data Science maturity to achieve success, as indicated in the retrieved information.",9
4,Llama-2-13b-chat-hf,"The response correctly illustrates the interdisciplinary nature of Data Science using the Data Science Venn Diagram, highlighting the overlap between computer science, mathematics, and domain expertise. Moreover, it accurately explains why these areas are critical in the field of Data Science. It presents the information in a clear and factual manner, aligning with the details provided in the reference answer. The response could have further emphasized on how each of these areas contributes to Data Science: domain expertise ensures relevance, computer science handles data manipulation, and mathematics provides the basis for analysis and prediction.",9
5,Mistral-7B-Instruct-v0.2,"The response correctly identifies the fundamental differences between deductive and inductive reasoning in the context of Data Science as outlined in the guide. It accurately describes deductive reasoning as proceeding from known premises to a certain conclusion and inductive reasoning as drawing uncertain inferences based on probabilistic reasoning. It also correctl

In [45]:
llm_results.groupby("llm_model")[["evaluation_score"]].mean()

,evaluation_score
llm_model,
AzureOpenai-GPT-35-turbo-16k,9.6
LargeWorldModel-LWM-Text-Chat-256K,8.4
Llama-2-13b-chat-hf,8.6
Mistral-7B-Instruct-v0.2,8.8
